In [8]:
from dotenv import load_dotenv
import os

In [10]:
load_dotenv() 
API_KEY = os.getenv("GOOGLE_API_KEY")

## TESTING

In [ ]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

#### Answer: 1

In [17]:
from google import genai
import json
import random


def get_weather(city: str) -> float:
    known_weather_data = {'berlin': 20.0}
    city = city.strip().lower()
    return known_weather_data.get(city, round(random.uniform(-5, 35), 1))

# Step 1: User query
question = "What's the weather like in Berlin?"

prompt = f"""
You're a helpful assistant with access to a tool called `get_weather(city: str) -> float`.

If someone asks about the weather in a city, respond with:
{{
  "action": "CALL",
  "function": "get_weather",
  "arguments": {{
    "city": "<city name>"
  }}
}}

Otherwise respond with a regular answer.

User: {question}
"""

# Step 2: Call Gemini
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[genai.types.Content(role="user", parts=[genai.types.Part(text=prompt)])]
)

reply = response.candidates[0].content.parts[0].text
print("Gemini raw reply:\n", reply)

# Step 3: Parse and execute function if valid JSON
start = reply.find('{')
end = reply.rfind('}') + 1

try:
    response_json = json.loads(reply[start:end])
    if response_json.get("action") == "CALL" and response_json.get("function") == "get_weather":
        city = response_json["arguments"]["city"]
        result = get_weather(city)
        print(f"✅ Function result: {result}°C")
    else:
        print("Gemini didn't suggest a function call. Response:", reply)
except Exception as e:
    print("Could not parse Gemini's reply as JSON. Response:", reply)
    print("Error:", str(e))


Gemini raw reply:
 ```json
{
  "action": "CALL",
  "function": "get_weather",
  "arguments": {
    "city": "Berlin"
  }
}
```

✅ Function result: 20.0°C


In [ ]:
known_weather_data = {
    "berlin": 20.0
}

def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

#### Answer: 2 

In [ ]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Adds or updates the weather temperature for a specific city in the database.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city to update weather for."
            },
            "temp": {
                "type": "number",
                "description": "Temperature in Celsius for the city."
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}
